In [30]:
#Packages
import pandas as pd

In [31]:
# Functions
def categorize_race(race):
    categories = {
        'White': ['White', 'American Indian or Alaska Native, White', 'Asian, White', 'Black or African American, White', 'Other: Middle Eastern', 'Unknown, not collected, White'],
        'Black/African American': ['Black or African American', 'Black or African American, White'],
        'Hispanic/Latinx': ['Other: Hispanic', 'Other: "Hispanic"', 'Other: Latino', 'Other: Guyanese', 'Other: Puerto Rican', 'Other: Latina', 'Other: hispanic', 'Other: Mexican'],
        'Asian/Pacific Islander': ['Asian', 'Other: East Indian', 'Asian, White', 'Other: Filipino', 'Native Hawaiian or Other Pacific Islander', 'Other: Indian'],
        'American Indian or Alaska Native': ['American Indian or Alaska Native', 'American Indian or Alaska Native, White'],
    }

    for category, labels in categories.items():
        if race in labels:
            return category
    return 'Other/Unknown'

In [32]:
# Load data
sirs_demo = pd.read_excel('../data/Dartmouth Data Set- SIRS .xlsx', sheet_name = 'Demographics')
sirs_abc = pd.read_excel('../data/Dartmouth Data Set- SIRS .xlsx', sheet_name = 'ABC Data')
feis = pd.read_excel('../data/Dartmouth FEIS Data.xlsx')

In [33]:
# Convert dates
sirs_start_date = sirs_demo['Date Enrolled in START'].min().strftime('%B %d, %Y')
sirs_end_date = sirs_demo['Date Enrolled in START'].max().strftime('%B %d, %Y')

sirs_abc_start_date = pd.to_datetime(sirs_abc['Date Reviewed'], errors='coerce').min().strftime('%B %d, %Y')
sirs_abc_end_date = pd.to_datetime(sirs_abc['Date Reviewed'], errors='coerce').max().strftime('%B %d, %Y')

feis_start_date = feis['Start Date'].min().strftime('%B %d, %Y')
feis_end_date = feis['Start Date'].max().strftime('%B %d, %Y')

In [34]:
# Descriptive analysis table for data
sirs_observations = sirs_demo.shape[0]

sirs_abc_observations = sirs_abc.shape[0]

feis_observations = feis.shape[0]


descriptive_analysis = pd.DataFrame({
    'Sheet Name': ['Aberrant Behavior Checklist (ABC)', 'Demographics', 'Family Experiences with Severe Mental Illness Scale (FEIS)'],
    'Number of Observations': [sirs_abc_observations, sirs_observations, feis_observations],
    'Date Range' : [sirs_abc_start_date + " to " + sirs_abc_end_date, sirs_start_date + " to " + sirs_end_date, feis_start_date + " to " + feis_end_date],
    'Unit of Analysis': ["Record of Mental Health Stability", "Individual START Participant", "Respondent Survey Response"]
})

styled_total = descriptive_analysis.style.hide(axis='index')\
    .format(na_rep='-', precision=0)\
    .set_table_styles(
        [{'selector': 'caption',
          'props': [('color', 'black'),
                    ('font-size', '16px'),
                    ('text-align', 'center'),
                    ('font-weight', 'bold')]},
         {'selector': 'th',
          'props': [('border', '1px solid black'),
                    ('background-color', '#f7f7f7'),
                    ('padding', '5px')]},
         {'selector': 'td',
          'props': [('border', '1px solid black'),
                    ('padding', '5px')]}]
    )

# Export to excel into our output folder
styled_total.to_excel('../output/START_Data_Analysis.xlsx', index=False)

In [35]:
# Getting race counts to normalize
race_value_counts = sirs_demo.Race.value_counts()
print(race_value_counts)

race_norm = sirs_demo['Race'].value_counts(normalize = True)

race_norm_cumsum = race_norm.cumsum().head(30)
print(race_norm_cumsum)

Race
White                                        2980
Black or African American                     994
Unknown, not collected                        277
Other: Hispanic                               191
Asian                                         147
                                             ... 
Other: Ecuadorian                               1
Other: Pakistani                                1
Other: Pakistani-Muslim                         1
Other: White and Black or African America       1
Other: mexican                                  1
Name: count, Length: 162, dtype: int64
Race
White                                                0.597673
Black or African American                            0.797032
Unknown, not collected                               0.852587
Other: Hispanic                                      0.890895
Asian                                                0.920377
Other                                                0.927597
Black or African American, Wh

In [36]:
# Taking top 96% of data
sirs_demo['race_clean'] = sirs_demo['Race'].apply(categorize_race)
sirs_demo.race_clean.value_counts()

race_clean
White                               3043
Black/African American               994
Other/Unknown                        491
Hispanic/Latinx                      264
Asian/Pacific Islander               167
American Indian or Alaska Native      27
Name: count, dtype: int64

In [37]:
sirs_merge = pd.merge(sirs_demo, sirs_abc, how = 'inner', left_on = 'Local ID', right_on = 'Local ID')

In [38]:
#printing df shape to make sure our merge was executed correctly
print(sirs_merge.shape)
print(sirs_demo.shape)
print(sirs_abc.shape)

(19461, 53)
(4986, 46)
(26056, 8)


In [39]:
sirs_merge.to_pickle('../data/sirs_demo_abc_clean')

In [40]:
# Drop duplicate Local ID values from the SIRS dataset
sirs_demo_unique = sirs_demo.drop_duplicates(subset='Local ID')

In [41]:
#printing df shape to make sure our drop was executed correctly

print(sirs_demo_unique.shape)
print(sirs_demo.shape)
print(feis.shape)

(4986, 46)
(4986, 46)
(1940, 57)


In [42]:
# Merging the datasets on 'Local ID'
merged_data = pd.merge(sirs_demo_unique, feis, left_on='Local ID', right_on = 'Respondent ID #  (SIRS Local ID)', how='inner')

In [43]:
#printing df shape to make sure our merge was executed correctly

print(sirs_demo_unique.shape)
print(sirs_demo.shape)
print(feis.shape)
print(merged_data.shape)

(4986, 46)
(4986, 46)
(1940, 57)
(1097, 103)


In [44]:
merged_data.to_pickle('../data/feis_sirs_demo_merged')